In [ ]:
# DBSCAN clustering 
import numpy as np
from sklearn.cluster import  DBSCAN
from sklearn.metrics import confusion_matrix, f1_score, normalized_mutual_info_score, precision_score, recall_score
import pandas as pd
import timeit
import matplotlib.pyplot as plt
import os 

In [ ]:
# the method specifies key to make sure the ground truth set is in consistent order as the valeus accessed 
def ground_truth_set(key):
    ground_truth = []
    for item in key:
        if "Palm" in item: 
            ground_truth.append(1)
        else:
            ground_truth.append(0)
    
    return np.array(ground_truth)

def data_set(key, dict):
    data = []
    for item in key:
        data.append(dict[item])
    return np.array(data)

def get_truth_and_data(fileName):
    dict = np.load(fileName, allow_pickle=True).item()
    dict_keys = dict.keys()
    ground_truth = ground_truth_set(dict_keys)
    data_values = data_set(dict_keys, dict)

    return ground_truth, data_values

def accuracy(truth, pred):
    tn, fp, fn, tp = confusion_matrix(truth, pred, labels=[0,1]).ravel()
    return (tn+tp)/ (tn+ fp+ fn+ tp)

def three_d_accuracy(truth, pred):
    matrix1 = confusion_matrix(truth, pred, labels=[-1,0,1])
    
    trues1 = matrix1[0][0]+matrix1[1][1]+matrix1[2][2]
    accuracy_percent1 = trues1/len(truth)

    #see if swapped around is better 
    mapping = {-1:-1, 0: 1, 1: 0}
    # Reassign the labels using the mapping
    new_pred = np.array([mapping[label] for label in pred])

    matrix2 = confusion_matrix(truth, new_pred, labels=[-1,0,1])
    
    trues2 = matrix2[0][0]+matrix2[1][1]+matrix2[2][2]
    accuracy_percent2 = trues2/len(truth)

    if (accuracy_percent1 > accuracy_percent2):
        print(matrix1)
        # print("1. ", accuracy_percent1)
        return accuracy_percent1
    # print("2. ", accuracy_percent2)
    print(matrix2)
    return accuracy_percent2


In [ ]:
glcm_truth_n , glcm_data_n  =  get_truth_and_data('GLCM_n.npy')
glcm_truth , glcm_data  =  get_truth_and_data('GLCM.npy')

hist_truth_n, hist_data_n =  get_truth_and_data('hist_n.npy')
hist_truth, hist_data =  get_truth_and_data('hist.npy')

hog_truth_n, hog_data_n =  get_truth_and_data('hog_n.npy')
hog_truth, hog_data =  get_truth_and_data('hog.npy')

lbp_truth_n, lbp_data_n =  get_truth_and_data('lbp_n.npy')
lbp_truth, lbp_data =  get_truth_and_data('lbp.npy')

In [ ]:
print(np.sum(lbp_truth == 0)) # 14.7%
print(np.sum(lbp_truth == 0))

In [ ]:
def dbscan_labels(e, dataset):
    minpt = int(len(dataset)*0.025)
    print("chosen minpt is 0.1 of sample size ", minpt)
    print("the chosen eps: ", e)
    dbscan = DBSCAN(eps = e, min_samples=minpt)
    labels = dbscan.fit_predict(dataset)
    labels = np.array(labels)
    unique = np.unique(labels)
    result = np.sum(labels == -1)
    print("The unique labels are: ", unique)
    print("the amount of noisy samples are: ", result)
    return labels
from sklearn.neighbors import NearestNeighbors

def knn_plot(data):
    nn = int(len(data)*0.025)-1
    neighbors = NearestNeighbors(n_neighbors=nn)
    neighbors_fit = neighbors.fit(data)
    distances, indices = neighbors_fit.kneighbors(data)

    distances = np.sort(distances[:, -1])  # Get the distances to the 4th nearest neighbor

    plt.plot(distances)
    plt.xlabel("Points sorted by distance")
    plt.ylabel("Distance to 4th nearest neighbor")
    plt.title("k-NN Distance Plot")
    plt.show()

def experiment(e, truth, data):
    labels = dbscan_labels(e, data)
    precision = precision_score(truth, labels, labels = [0,1], average = 'weighted')
    recall = recall_score(truth, labels, labels = [0,1], average = 'weighted')
    f1 = f1_score(truth, labels, labels = [0,1], average = 'weighted')
    return precision, recall, f1 

import csv 
def record_results(e, truth, data, dict, name, experi ="dbscan"): 
    precision, recall, f1 = experiment(e, truth,data)
    if name not in dict:
        dict[name] = [precision, recall, f1]

    if not os.path.exists(f"results_{experi}/{name}.csv"):
        with open(f"results_{experi}/{name}.csv", 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(("precision","recall", "f1"))
            writer.writerow((precision, recall, f1))
    else:
        with open(f"results_{experi}/{name}.csv", 'a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow((precision, recall, f1)) 
    
    print(name)
    if dict[name][0] <= precision:
        dict[name][0] = precision
    else:
        print("max precision: ", dict[name][0])

    if dict[name][1] <= recall:
        dict[name][1] = recall
    else:
        print("max recall: ", dict[name][0])

    if dict[name][2] <= f1:
        dict[name][2] = f1
    else:
        print("max f1 : ", dict[name][2])
    
    print("f1 : ", f1 , "\nprecision: ", precision,"\nrecall: ", recall)
    

def experiment_eps(title, eps_array, truth, dataset):
    print(title)
    for i in eps_array:
        labels = dbscan_labels(i, dataset)
        arr = np.array(labels)
        unique = np.unique(labels)
        result = np.sum(arr == -1)
        
        if(len(unique) == 3 or len(unique) == 2 or len(unique) == 4 or len(unique) == 5):
            f1 = f1_score(truth, labels, labels = [0,1], average = 'weighted')
            print("F1 score: ", f1)
            # print("accuracy: ", three_d_accuracy(truth, labels))
        print()



In [5]:
knn_plot(hog_data_n)

In [ ]:
arr = np.linspace(4.81, 4.9, num=10) # results from 3 minpoints 
experiment_eps("hog n data", arr, hog_truth_n, hog_data_n)

# chosen minpt is 0.1 of sample size  66
# the chosen eps:  4.81
# The unique labels are:  [-1  0  1]
# the amount of noisy samples are:  1593
# F1 score:  0.028622279390097045


In [ ]:
knn_plot(hog_data)

In [ ]:
arr = np.linspace(3.251, 3.26, num=10)

experiment_eps("hog data", arr, hog_truth, hog_data)

# chosen minpt is 0.1 of sample size  66
# the chosen eps:  3.259
# The unique labels are:  [-1  0  1]
# the amount of noisy samples are:  2370
# F1 score:  0.09413519100886412


In [ ]:
knn_plot(glcm_data_n)

In [ ]:
arr = np.linspace(0.071, 0.09, num=20)
experiment_eps("glcm n data", arr, glcm_truth_n , glcm_data_n)

# chosen minpt is 0.1 of sample size  66
# the chosen eps:  0.076
# The unique labels are:  [-1  0  1]
# the amount of noisy samples are:  2514
# F1 score:  0.04726882976564941


In [ ]:
knn_plot(glcm_data)

In [ ]:
arr = np.linspace(291,295, num=10)
experiment_eps("glcm data", arr, glcm_truth , glcm_data)

# chosen minpt is 0.1 of sample size  66
# the chosen eps:  294.1111111111111
# The unique labels are:  [-1  0  1]
# the amount of noisy samples are:  1729
# F1 score:  0.1045771853001049


In [ ]:
knn_plot(hist_data_n)

In [ ]:
#arr = np.linspace(201,400, num=200)
arr = np.linspace(0.01,4, num=40)
experiment_eps("hist n data", arr, hist_truth_n, hist_data_n)

# chosen minpt is 0.1 of sample size  66
# the chosen eps:  1.34
# The unique labels are:  [-1  0  1]
# the amount of noisy samples are:  74
# F1 score:  0.6974513447274793

In [ ]:
knn_plot(hist_data)

In [ ]:
arr = np.linspace(0.0041,0.005, num=10)
experiment_eps("hist data", arr, hist_truth, hist_data)

# chosen minpt is 0.1 of sample size  66
# the chosen eps:  0.004200000000000001
# The unique labels are:  [-1  0  1]
# the amount of noisy samples are:  2295
# F1 score:  0.04244043330528836


In [ ]:
knn_plot(lbp_data)

In [ ]:
arr = np.linspace(0.002,0.011, num=10)
experiment_eps("lbp data", arr, lbp_truth, lbp_data)

# chosen minpt is 0.1 of sample size  66
# the chosen eps:  0.01
# The unique labels are:  [-1  0  1]
# the amount of noisy samples are:  2525
# F1 score:  0.05038290304924796

In [ ]:
knn_plot(lbp_data_n)

In [ ]:
arr = np.linspace(0.0096,0.01, num=100)
arr = [0.009701, 0.009702, 0.009703, 0.009704, 0.009705, 0.009706, 0.009707, 0.009708]
experiment_eps("lbp data", arr, lbp_truth_n, lbp_data_n)

# chosen minpt is 0.1 of sample size  66
# the chosen eps:  0.0097010101010101
# The unique labels are:  [-1  0  1]
# the amount of noisy samples are:  2535
# F1 score:  0.038449183583668375

# chosen minpt is 0.1 of sample size  66
# the chosen eps:  0.00971
# The unique labels are:  [-1  0  1]
# the amount of noisy samples are:  2535
# F1 score:  0.03772670069896626


# chosen minpt is 0.1 of sample size  66
# the chosen eps:  0.00974
# The unique labels are:  [-1  0  1]
# the amount of noisy samples are:  2534
# F1 score:  0.03772232242870149


# chosen minpt is 0.1 of sample size  66
# the chosen eps:  0.009749494949494949
# The unique labels are:  [-1  0  1]
# the amount of noisy samples are:  2533
# F1 score:  0.03844040552829722



# chosen minpt is 0.1 of sample size  66
# the chosen eps:  0.00975
# The unique labels are:  [-1  0  1]
# the amount of noisy samples are:  2518
# F1 score:  0.04914017133210346



In [ ]:
p_r_f1 ={}

In [ ]:
# official min points 
record_results(4.81, hog_truth_n , hog_data_n,  p_r_f1,   "hog_n" )
record_results(3.259, hog_truth , hog_data,  p_r_f1,   "hog" )

record_results(0.076, glcm_truth_n , glcm_data_n,  p_r_f1,   "glcm_n" )
record_results(294, glcm_truth , glcm_data,  p_r_f1,   "glcm" )

record_results(1.34, hist_truth_n , hist_data_n,  p_r_f1,   "hist_n" )
record_results(0.042, hist_truth , hist_data,  p_r_f1,   "hist" )


record_results(0.01, lbp_truth , lbp_data,  p_r_f1,   "lbp" )
record_results( 0.00975, lbp_truth_n , lbp_data_n,  p_r_f1,   "lbp_n" )

In [ ]:
# results from minpt at 3 
record_results(3.655, hog_truth , hog_data,  p_r_f1,   "hog" )
record_results(3.78, hog_truth_n , hog_data_n,  p_r_f1,   "hog_n" )
record_results(613.5, glcm_truth , glcm_data,  p_r_f1,   "glcm" )
record_results(0.05, glcm_truth_n , glcm_data_n,  p_r_f1,   "glcm_n" )
record_results(374, hist_truth_n , hist_data_n,  p_r_f1,   "hist_n" )
record_results(373, hist_truth , hist_data,  p_r_f1,   "hist" )
record_results(0.01789, lbp_truth , lbp_data,  p_r_f1,   "lbp" )

# lbp
# f1 :  0.03706204445867489 
# precision:  0.5898159177581467 
# recall:  0.14234330756868446

#record_results( , lbp_truth_n , lbp_data_n,  p_r_f1,   "lbp_n" )

In [ ]:
record_results(306, glcm_truth , glcm_data,  p_r_f1,   "glcm" )

In [ ]:
# appendix section 
record_results(306, glcm_truth , glcm_data,  p_r_f1,   "glcm" )
record_results(3, hist_truth_n , hist_data_n,  p_r_f1,   "hist_n" )


record_results(4.81, hog_truth_n , hog_data_n,  p_r_f1,   "hog_n" )
record_results(3.259, hog_truth , hog_data,  p_r_f1,   "hog" )

record_results(0.076, glcm_truth_n , glcm_data_n,  p_r_f1,   "glcm_n" )


record_results(373, hist_truth , hist_data,  p_r_f1,   "hist" )
record_results(0.01789, lbp_truth , lbp_data,  p_r_f1,   "lbp" )
record_results( 0, lbp_truth_n , lbp_data_n,  p_r_f1,   "lbp_n" )